In [1]:
# coding:utf-8

import keras
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import array_to_img, img_to_array, list_pictures, load_img
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from keras import backend as K
import os
import glob
import matplotlib.gridspec as gridspec
import shutil
from keras.utils import plot_model
import cv2
from skimage import io
from skimage import transform
from scipy.misc import imresize
from skimage import io
import skimage

Using TensorFlow backend.


In [2]:
# def draw_images(datagen, x, result_images):
#     # 出力先ディレクトリを作成
#     temp_dir = "ll"
#     shutil.rmtree(temp_dir)
#     os.mkdir(temp_dir)

#     # generatorから9個の画像を生成
#     # xは1サンプルのみなのでbatch_sizeは1で固定
#     g = datagen.flow(x, batch_size=1, save_to_dir=temp_dir, save_prefix='jpg', save_format='jpg')
#     g.show()
#     for i in range(9):
#         batch = g.next()

#     # 生成した画像を3x3で描画
#     images = glob.glob(os.path.join(temp_dir, "*.jpg"))
#     fig = plt.figure()
#     gs = gridspec.GridSpec(3, 3)
#     gs.update(wspace=0.01, hspace=0.01)
#     print(len(images))
#     for i in range(9):
#         img = load_img(images[i])
#         plt.subplot(gs[i])
#         plt.imshow(img, aspect='auto')
# #         plt.axis("off")
#         plt.show()
#     plt.savefig(result_images)

#     # 出力先ディレクトリを削除
# #     shutil.rmtree(temp_dir)

In [3]:
# x = img_to_array(load_img('./data/train/carot/0.jpg', target_size=(64,64)))
# print(x)
# x = np.expand_dims(x, axis=0)
# datagen = ImageDataGenerator(rotation_range=90)
# draw_images(datagen, x, './data/train/carot/test.jpg')


In [4]:
# フォルダの中にある画像を順次読み込む
# カテゴリーは0から始める

#参考
#https://qiita.com/supersaiakujin/items/fc54116df9ca6958a68d
#http://testpy.hatenablog.com/entry/2017/06/02/001901
#https://qiita.com/zaburo/items/0b9db87d0a52191b164b
# 画像の反転と回転の実装、またはdata argumentation の実装を行う
X = []
Y = []

#下、後でコード量を減らす
# 対象Aの画像
rate = 0.2
label = 0
for dir in os.listdir("data/train"):
    print(dir)
    if dir == ".DS_Store":
        continue
    for picture in list_pictures("data/train/"+dir):
#         print(picture)
    #     img =load_img(picture, target_size=(64,64))
        img =imresize(skimage.io.imread(picture),(64,64), interp='nearest')
    #     img_raw = skimage.io.imread
        for i in range(350,360):
            #回転のみ
            X.append(img_to_array(skimage.transform.rotate(img, angle=i, resize=False, center=None)))
            Y.append(label )
            #回転と上下の反転
            X.append(img_to_array(cv2.flip(skimage.transform.rotate(img, angle=i, resize=False, center=None),0)))
            Y.append(label )
            #回転と左右の反転
            X.append(img_to_array(cv2.flip(skimage.transform.rotate(img, angle=i, resize=False, center=None),1)))
            Y.append(label )
            #拡大
            X.append(img_to_array(skimage.transform.warp(skimage.transform.rotate(img, angle=i, resize=False, center=None), skimage.transform.AffineTransform(scale=(1-rate, 1-rate), translation=(64,64)))))
            Y.append(label )
    label += 1

# # arrayに変換
X = np.asarray(X)
# print(X)
Y = np.asarray(Y)
# print(Y)

.DS_Store
carot
CurryPowder
onion
potato
rice


In [5]:
# 画素値を0から1の範囲に変換
X = X.astype('float32')
X = X / 255.0

# クラスの形式を変換
Y = np_utils.to_categorical(Y, 5)

# 学習用データとテストデータ
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=111)

In [ ]:
# CNNを構築
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))       # クラスは2個
model.add(Activation('softmax'))

# コンパイル
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

# 実行。出力はなしで設定(verbose=0)。
history = model.fit(X_train, y_train, batch_size=5, epochs=300,
                   validation_data = (X_test, y_test), verbose = 1)

# plot_model(model, to_file="model.png", show_shapes=True)
# https://qiita.com/agumon/items/ab2de98a3783e0a93e66

Train on 6912 samples, validate on 768 samples
Epoch 1/300

 480/6912 [=>............................] - ETA: 5:51 - loss: 1.5845 - acc: 0.2458


 965/6912 [===>..........................] - ETA: 5:08 - loss: 1.5774 - acc: 0.2456


1450/6912 [=====>........................] - ETA: 4:39 - loss: 1.5774 - acc: 0.2607


1935/6912 [=======>......................] - ETA: 4:08 - loss: 1.5698 - acc: 0.2724


2420/6912 [=========>....................] - ETA: 3:42 - loss: 1.5642 - acc: 0.2711


...

1355/6912 [====>.........................] - ETA: 9:14 - loss: 1.5522 - acc: 0.2686


1840/6912 [======>.......................] - ETA: 8:27 - loss: 1.5497 - acc: 0.2587


2325/6912 [=========>....................] - ETA: 7:09 - loss: 1.5554 - acc: 0.2598


2810/6912 [===========>..................] - ETA: 5:59 - loss: 1.5542 - acc: 0.2633


2985/6912 [===========>..................] - ETA: 5:35 - loss: 1.5510 - acc: 0.2653

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['acc', 'val_acc'], loc='lower right') #グラフの凡例を用意している
plt.show()

In [ ]:
# テストデータに適用
predict_classes = model.predict_classes(X_test)

# マージ。yのデータは元に戻す
mg_df = pd.DataFrame({'predict': predict_classes, 'class': np.argmax(y_test, axis=1)})

# confusion matrix
pd.crosstab(mg_df['class'], mg_df['predict'])

In [ ]:
def train_and_evaluate_model(model,data,labels,data_test,labels_test):
# def train_and_evaluate_model(model, data[train], labels[train], data[test], labels[test]):
    model.fit(data, labels, batch_size=32, epochs=30,verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation
    return model.evaluate(data_test, labels_test, verbose=1)  # Evaluate the trained model on the test set!
#損失値と評価の値を算出している
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import StratifiedKFold
# kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=1)
# print(cross_val_score(model, image_list, label_list,cv=kfold)).mean()  
from sklearn.cross_validation import StratifiedKFold
label = np.r_[np.repeat(0,16), np.repeat(1,0)]
skf = StratifiedKFold(label, n_folds=15, shuffle=False)
total_score = 0
rep = 0
print("success")
for i, (train, test) in enumerate(skf):
    print("TRAIN:", train, "TEST:", test)
    total_score += train_and_evaluate_model(model, X_train[train], y_train[train], X_train[test], y_train[test])[1]
    rep+=1
# print(total_score/rep)
print("%.10f" % (total_score/rep))